In [1]:
import os
from cmdstanpy import CmdStanModel
import numpy as np
import pandas as pd

/home/laura/miniconda3/envs/abstraction/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Homework 1: Bayesian Cognitive and Rational Speech Act models

This homework assignment is to be completed in groups. It is due on November 27, 2025 (midnight). Please upload *all files you created or modified* to the homework folder of your group in studIP.

Group number:

Names:

*General note: It is permitted to use AI tools for coding. Please refer to the uploaded manual `AI_Tools_Guidelines` for recommended ways how to use AI to advance your studies in a way that supports your learning. That means that you should not be satisfied if an AI tool hands you a working version of your code, but that you should put in effort to understand how exactly the problem is solved. Another note of caution: What might work for large programming languages like Python, does not necessarily work for Stan. Check your code carefully and do NOT blindly trust AI.*

## Introduction
During the past weeks, you have learned how Bayesian inference works and how it can be used in Bayesian cognitive models. You also learned about a specific type of Bayesian models that can be used to model pragmatic language understanding and production, the Rational Speech Act models. The goal of this homework assignment is for you to learn how to implement Bayesian models in Stan and, specifically, how to implement RSA models in Stan. A special focus will be on the different use cases and evaluation methods of RSA models.

## 1) Stan modeling (8 points)

1.1) In the file `simple_model.stan`, you will find a simple Stan model. Describe its implementation, relating it to the knowledge you gained about the conventions for coding models in Stan. (4 points)

- The data block defines the observed data: we have N trials, and for each trial we know the semantic similarity, the phonological similarity, and the response time.
- In the transformed data block, the response times are log-transformed, so the model works on log_rt. This makes the normality assumption more reasonable, because raw response times are positive and often skewed.
- In the parameters block, the model estimates:
    - alpha (intercept),
    - beta_sem (effect of semantic similarity),
    - beta_phon (effect of phonological similarity),
    - sigma_rt (residual standard deviation, constrained to be positive).
    - Each of these parameters has a weakly informative normal prior.
- In the model block, each log_rt[i] is assumed to follow a normal distribution with mean alpha + beta_sem * semantic_similarity[i] + beta_phon * phonological_similarity[i] and standard deviation sigma_rt. So the two similarity measures linearly predict log response time.
- In the generated quantities block:
    - log_likelihood[i] stores the pointwise log density for each trial,
    - log_rt_ppd[i] stores posterior-predictive draws on the log scale,
    - rt_ppd[i] exponentiates those draws back to the original time scale, giving predicted response times for posterior predictive checks.

Essentially, it’s a Bayesian linear regression model that predicts how fast someone responds based on how semantically and phonologically related the word and context are.

1.2) You will notice that the model does not compile. Fix the problems and explain what you did. (4 points)

    The model compiles:)

## 2) Bayesian cognitive models (10 points)
Think of a use case for `simple_model.stan` in the scope of Bayesian cognitive modeling. Describe the model while answering the following questions:

2.1) What cognitive capacity can be explained by this model? (2 points)

- The model describes lexical access in tasks like visual lexical decision or picture naming: how quickly a person can retrieve the correct word form when given semantic and phonological cues from a context.
- The predictors capture two psychologically meaningful influences:
    - semantic priming: effect of meaning similarity,
    - phonological priming: effect of sound similarity,
- and the strength of these influences changes the observed response times.

2.2) What is the purpose and function of this model? (3 points)

- The model explains how semantic and phonological similarity jointly produce the distribution of RTs we observe.
- By fitting this Bayesian regression, we can:
    - estimate how much semantic and phonological similarity speed up or slow down retrieval,
    - quantify priming effects at the level of participants or conditions,
    - generate posterior predictive RT distributions for new items or contexts,
    - compare hypotheses (e.g., “semantic effects are stronger than phonological effects”) using the posterior over the beta coefficients.

2.3) At which level of analysis does it model this cognitive capacity and why? (3 points)

- The Stan model is a computational model in Marr’s framework. It specifies what mapping the cognitive system implements: given inputs (semantic similarity and phonological similarity), it defines a probability distribution over response times.
- The code does not specify an algorithm or representation format for lexical access (no activation dynamics, no specific retrieval steps, no architecture), so it is not an algorithmic-level model. And it does not mention neural structures or hardware, so it is not an implementational-level model either. It remains at the abstract, probabilistic computational-level description of the capacity “map semantic/phonological similarity to response-time distributions.”


Overall coherence gives another 2 points.

## 3) RSA modeling (82 points)
The purpose of the following model is to explain the use of overinformative referring expressions in pragmatic communication. In referential communication, the speaker’s task is to produce a referring expression that allows a listener to identify the target in the context. Consider the context below, where the target is the small blue pin. A referring expression including a size adjective (the small pin) is strictly speaking sufficient for uniquely establishing reference to the target, yet speakers often “overmodify” with color, producing referring expressions like the small blue pin. This overmodification phenomenon is what the model is intended to capture.

<img src="img/size-sufficient.png" width="400"/>

### 3.1) Vanilla RSA (20 points)
In the file `vanilla_rsa.stan`, you find an RSA model of the production of referring expressions, based on the vanilla RSA model of Frank & Goodman (2012) that we discussed in class.

3.1.1) Provide informative comments in the file `vanilla_rsa.stan`. (4 points)

3.1.2) You will notice that the parameters and model blocks are empty. Why is that? Go through the following code and inspect the model's behavior. Look at the stan variables that are included in the fitted model. (3 points)

    There are no parameters and no defined model, as we are not doing Bayesian sampling. We are not trying to estimate any values but rather arrive at our results deterministically by doing matrix transformations within transformed parameters{}.

    We pass our data to stan as a dictionary. It is important to note that stan does not accept matrices and arrays, so these have to be transformed into lists first.

In [115]:
# compile model
stan_file = os.path.join('stan', 'vanilla_rsa.stan')
rsa_model = CmdStanModel(stan_file=stan_file)

23:48:15 - cmdstanpy - INFO - compiling stan file C:\Users\User\Proton Drive\maa_ng0h\My files\C. Work\2E Modeling Communication and Abstraction\Studyproject_HW1\stan\vanilla_rsa.stan to exe file C:\Users\User\Proton Drive\maa_ng0h\My files\C. Work\2E Modeling Communication and Abstraction\Studyproject_HW1\stan\vanilla_rsa.exe
23:48:36 - cmdstanpy - INFO - compiled model executable: C:\Users\User\Proton Drive\maa_ng0h\My files\C. Work\2E Modeling Communication and Abstraction\Studyproject_HW1\stan\vanilla_rsa.exe
23:48:36 - cmdstanpy - INFO - compiled model executable: C:\Users\User\Proton Drive\maa_ng0h\My files\C. Work\2E Modeling Communication and Abstraction\Studyproject_HW1\stan\vanilla_rsa.exe


In [116]:
# define input data
states = ["big_blue", "big_red", "small_blue"]
utterances = [
    "big", "small", "blue", "red"
]

n_states = len(states)
n_utterances   = len(utterances)

# build meaning_matrix[u, s]
meaning_matrix = np.zeros((n_utterances, n_states), dtype=int)
for u, utterance in enumerate(utterances):
    for s, state in enumerate(states):
        # literal meaning maps to true iff the utterance string appears in the state string
        # Stan cannot handle booleans, so we need to work with integers here
        meaning_matrix[u, s] = int(utterance in state)

# parameters - change them here
alpha = 1.0
cost_weight = 1.0

# cost function
cost_dict = {
    "big": 0.0,
    "small": 0.0,
    "blue": 0.0,
    "red": 0.0,
}
cost = np.array([cost_dict[utterance] for utterance in short_utterances])


# prepare Stan data as dictionary
stan_data = {
    "S": n_states,
    "U": n_utterances,
    "meaning_matrix": meaning_matrix.tolist(), # Stan cannot handle numpy arrays       
    "cost": cost.tolist(),                     # or dictionaries
    "alpha": alpha,
    "cost_weight": cost_weight
}

fit = rsa_model.sample(stan_data, show_console=True, chains=1, iter_warmup=300, adapt_engaged=True, iter_sampling=300)

23:48:51 - cmdstanpy - INFO - Chain [1] start processing
23:48:51 - cmdstanpy - INFO - Chain [1] done processing
23:48:51 - cmdstanpy - INFO - Chain [1] done processing


Chain [1] method = sample (Default)
Chain [1] sample
Chain [1] num_samples = 300
Chain [1] num_warmup = 300
Chain [1] save_warmup = 0 (Default)
Chain [1] thin = 1 (Default)
Chain [1] adapt
Chain [1] engaged = 1 (Default)
Chain [1] gamma = 0.050000000000000003 (Default)
Chain [1] delta = 0.80000000000000004 (Default)
Chain [1] kappa = 0.75 (Default)
Chain [1] t0 = 10 (Default)
Chain [1] init_buffer = 75 (Default)
Chain [1] term_buffer = 50 (Default)
Chain [1] window = 25 (Default)
Chain [1] algorithm = hmc (Default)
Chain [1] hmc
Chain [1] engine = nuts (Default)
Chain [1] nuts
Chain [1] max_depth = 10 (Default)
Chain [1] metric = diag_e (Default)
Chain [1] metric_file =  (Default)
Chain [1] stepsize = 1 (Default)
Chain [1] stepsize_jitter = 0 (Default)
Chain [1] num_chains = 1 (Default)
Chain [1] id = 1 (Default)
Chain [1] data
Chain [1] file = C:\Users\User\AppData\Local\Temp\tmpg4mr39y9\5fo9opp4.json
Chain [1] init = 2 (Default)
Chain [1] random
Chain [1] seed = 98410
Chain [1] outpu

In [117]:

L0_draws = fit.stan_variable('L0')
S1_draws = fit.stan_variable('S1')

L0 = L0_draws[0]
S1 = S1_draws[0]

df_L0 = pd.DataFrame(L0, index=utterances, columns=states)
df_S1 = pd.DataFrame(S1, index=states, columns=utterances)

print("L0:")
print(df_L0.to_string(float_format="{:.2f}".format))
print("S1:")
print(df_S1.to_string(float_format="{:.2f}".format))

L0:
       big_blue  big_red  small_blue
big        0.50     0.50        0.00
small      0.00     0.00        1.00
blue       0.50     0.00        0.50
red        0.00     1.00        0.00
S1:
            big  small  blue  red
big_blue   0.50   0.00  0.50 0.00
big_red    0.33   0.00  0.00 0.67
small_blue 0.00   0.67  0.33 0.00


3.1.3) Are the outputs in line with what you would expect given your knowledge about pragmatic communication and overinformative referring expressions?
Add complex utterances to the model (i.e., utterance consisting of a size and color adjective) and inspect the output again. The meaning of a complex two-word utterance is defined with intuitive intersective semantics: $$\mathcal{L}(u_{\text{complex}}, o)=\mathcal{L}(u_{\text{size}},o)\times\mathcal{L}(u_{\text{color}},o)$$ (6 points)

    I think the outputs are matching my intitions about pragmatic communication. We can see that the utterance "red" has a higher average utility value than "blue". This is expected, because the there is only one state ("big_red") that can correspond with "red" while blue could refer to two states ("big_blue", "small_blue").
    
    When adding a complex utterance where its consituence are found in the states but not in that combination (e.g. small_red), it's corresponding row in L0 contains NaN and consequentially the entire S1 matrix is filled with NaN. 
    
    If we add the three complex utterances that fully describe the states, we can observe that in S1, the utility of complex utterances is higher than the non-complex utterances, as long as there is no production cost that punishes utterance length. This makes intuitive sense, as complex utterances are more informatively describing the state than non-complex ones.

3.1.4) Play around with the rationality and cost weight parameters. How do they affect the model output? (4 points)

    Negative Values are prohibited, it throws an error.
    The lower rationality, the more uniform are the values distributed. 
    The higher the costs, the more non-complex statements are valued.
    Further, I created dynamic cost functions, that determine uttarance cost based on word length or letter length. With a positive letter length cost, the utterance "red" has lower cost than the utterance "blue". 

3.1.5) Adapt the utterance cost in a way that achieves a preference for overinformative referring expressions. (2 points)

    Setting Cost weight to 0 is one way to increase overinformativeness. Another is making the "per_letter" or "per_underscore" values in the custom cost functions negative, so that short expressions are effectively punished.

3.1.6) Adapt the utterance cost in a way that seems most natural to you. (1 point)

    I decided to use the custom "cost_per_underscore" function, as it dynamically calculates costs for each utterance based on word count. It is both pragmatic and natural, compared to the second custom cost function that weights based on letter count.. Further, I chose moderate cost_weight and per_underscore, because I the cost of typing or understanding another word in this situation is not very high. 

In [118]:
# define input data
states = ["big_blue", "big_red", "small_blue"]

utterances = [
    "big", "small", "blue", "red", "big_red", "big_blue", "small_blue"
]

n_states = len(states)
n_utterances   = len(utterances)

# build meaning_matrix[u, s]
meaning_matrix = np.zeros((n_utterances, n_states), dtype=int)
for u, utterance in enumerate(utterances):
    for s, state in enumerate(states):
        # literal meaning maps to true iff the utterance string appears in the state string
        # Stan cannot handle booleans, so we need to work with integers here
        meaning_matrix[u, s] = int(utterance in state)

# parameters - change them here
alpha = 0.8
cost_weight = 0.3

# new cost functions
def cost_by_letters(utterance, per_letter=0.1):
    letters = sum(1 for ch in utterance if ch.isalpha())
    return letters * per_letter

def cost_by_underscores(utterance, per_underscore=1.0):
    return utterance.count('_') * per_underscore

cost = np.array([cost_by_underscores(utterance) for utterance in utterances])

# prepare Stan data as dictionary
stan_data = {
    "S": n_states,
    "U": n_utterances,
    "meaning_matrix": meaning_matrix.tolist(), # Stan cannot handle numpy arrays       
    "cost": cost.tolist(),                     # or dictionaries
    "alpha": alpha,
    "cost_weight": cost_weight
}

fit = rsa_model.sample(stan_data, show_console=False, chains=1, iter_warmup=300, adapt_engaged=True, iter_sampling=300)

23:49:19 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

23:49:19 - cmdstanpy - INFO - CmdStan done processing.


In [119]:

L0_draws = fit.stan_variable('L0')
S1_draws = fit.stan_variable('S1')

L0 = L0_draws[0]
S1 = S1_draws[0]

df_L0 = pd.DataFrame(L0, index=utterances, columns=states)
df_S1 = pd.DataFrame(S1, index=states, columns=utterances)

print("L0:")
print(df_L0.to_string(float_format="{:.2f}".format))
print("S1:")
print(df_S1.to_string(float_format="{:.2f}".format))

L0:
            big_blue  big_red  small_blue
big             0.50     0.50        0.00
small           0.00     0.00        1.00
blue            0.50     0.00        0.50
red             0.00     1.00        0.00
big_red         0.00     1.00        0.00
big_blue        1.00     0.00        0.00
small_blue      0.00     0.00        1.00
S1:
            big  small  blue  red  big_red  big_blue  small_blue
big_blue   0.30   0.00  0.30 0.00     0.00      0.39        0.00
big_red    0.25   0.00  0.00 0.43     0.32      0.00        0.00
small_blue 0.00   0.43  0.25 0.00     0.00      0.00        0.32


### 3.2) Relaxed semantics (20 points)
It seems that our intuitions do not align well with the model. Let's use continuous rather than boolean semantics to see whether this can solve our problem. In the following, you need to adapt the RSA model and input data in a way that implements continuous semantics. The only change will be that the lexicon, or meaning matrix, should return real values instead of true or false: $$\mathcal{L}(u,o)\in [0,1] \subset \mathbb{R}$$
This approach captures the intuition that an object is not unambiguously big or blue, but rather that objects can count as big or blue to varying degrees.

3.2.1) Build a meaning matrix that captures the relaxed semantics with two new parameters size_semantics $x_\text{size}$ and color_semantics $x_\text{color}$. When an object $o$ is in the extension of a size adjective under the Boolean semantics defined above, take $\mathcal{L}(u,o)=x_\text{size}$, else $\mathcal{L}(u,o)=1-x_\text{size}$. The semantics are defined analogously for color. (6 points)
3.2.2) Run the model with alpha = 30, size_semantics = 0.8 and color_semantics = 0.99. Inspect the model outputs. (4 points)
3.2.3) Visualize the results of varying values for size_semantics and color_semantics, pit them against each other and interpret them. (6 points)
3.2.4) Van Gompel et al. (2019) found that speakers use overinformative referring expressions in about 80% of the trials that look like the one above, where size is sufficient to mention. What about contexts where color is sufficient to mention? Construct a context where color is sufficient to mention and interpret the output. (4 points)

In [6]:
# compile model
stan_file = os.path.join('stan', 'vanilla_rsa_relaxed_semantics.stan')
rsa_model = CmdStanModel(stan_file=stan_file)

In [7]:
# This is the code for exercise 3.2, adapted from the code for exercise 3.1.

# new parameters
x_size = 0.8 
x_color = 0.99

# define input data
states = ["big_blue", "big_red", "small_blue"]
utterances = [
    "big", "small", "blue", "red"
]
n_states = len(states)
n_utterances   = len(utterances)

# introduce size/color categorization
size_utterances  = ["big", "small"]
color_utterances = ["blue", "red"]

# build meaning_matrix[u, s]
meaning_matrix = np.zeros((n_utterances, n_states))
for u, utterance in enumerate(utterances):
    for s, state in enumerate(states):
        # check if object is in the extension of an adjective under Boolean semantics
        extension = utterance in state
        # check utterance category
        if utterance in size_utterances:
            x = x_size
        elif utterance in color_utterances:
            x = x_color
        # if object is in the extension of an adjective, take L(u,o) = x_size/color, else L(u,o) = 1 - x_size/color
        # e.g. "big" refers to big objects with probabiliy x and small objects with probability 1-x
        meaning_matrix[u, s] = x if extension else (1 - x)

# parameters - change them here
alpha = 30
cost_weight = 1.0

# cost function
cost_dict = {
    "big": 0.0,
    "small": 0.0,
    "blue": 0.0,
    "red": 0.0,
}
cost = np.array([cost_dict[utterance] for utterance in utterances])

# prepare Stan data as dictionary
stan_data = {
    "S": n_states,
    "U": n_utterances,
    "meaning_matrix": meaning_matrix.tolist(), # Stan cannot handle numpy arrays       
    "cost": cost.tolist(),                     # or dictionaries
    "alpha": alpha,
    "cost_weight": cost_weight,
    "x_size": x_size,
    "x_color": x_color
}

In [8]:
# Exercise 3.2.2: Run the model with alpha = 30, size_semantics = 0.8 and color_semantics = 0.99. Inspect the model outputs.

# We observe that color behaves like in non-relaxed semantics. For L0, P("big_red"|"red") = 1.0, reflecting that the color word
# practically determines that a pin of its color is chosen. We say "practically" because 0.99 is of course not 100%, but in
# practice we get a 1.0 probability. Again for L0, P("big_blue"|"blue") = 0.50. When the color word simply deterministically 
# decides that a pin of the stated color is to be chosen, the pins of the stated color are the only, equally viable candidates.
# S1 behaves analoguously. The size parameter is large enough to behave similarly as well.

# The number of chains should not make a difference with fixed parameters.
# For iter_warmup and iter_sampling, it is apparently best practice in a lot of cases to simply choose 500 because
# models will converge between 200 and 400. Going above 200 made absolutely no difference for us though, using a forward model.
fit = rsa_model.sample(stan_data, show_console=True, chains=1, iter_warmup=200, adapt_engaged=True, iter_sampling=200)
# fit.summary()

18:52:02 - cmdstanpy - INFO - Chain [1] start processing
18:52:02 - cmdstanpy - INFO - Chain [1] done processing


Chain [1] method = sample (Default)
Chain [1] sample
Chain [1] num_samples = 200
Chain [1] num_warmup = 200
Chain [1] save_warmup = 0 (Default)
Chain [1] thin = 1 (Default)
Chain [1] adapt
Chain [1] engaged = 1 (Default)
Chain [1] gamma = 0.050000000000000003 (Default)
Chain [1] delta = 0.80000000000000004 (Default)
Chain [1] kappa = 0.75 (Default)
Chain [1] t0 = 10 (Default)
Chain [1] init_buffer = 75 (Default)
Chain [1] term_buffer = 50 (Default)
Chain [1] window = 25 (Default)
Chain [1] algorithm = hmc (Default)
Chain [1] hmc
Chain [1] engine = nuts (Default)
Chain [1] nuts
Chain [1] max_depth = 10 (Default)
Chain [1] metric = diag_e (Default)
Chain [1] metric_file =  (Default)
Chain [1] stepsize = 1 (Default)
Chain [1] stepsize_jitter = 0 (Default)
Chain [1] num_chains = 1 (Default)
Chain [1] id = 1 (Default)
Chain [1] data
Chain [1] file = C:\Users\Hannes\AppData\Local\Temp\tmpfz_r_e0l\ck6t1zzj.json
Chain [1] init = 2 (Default)
Chain [1] random
Chain [1] seed = 73760
Chain [1] out

In [10]:
# Visualize results for exercise 3.2.2. 
# Exact same code as we used for exercise 3.1

L0_draws = fit.stan_variable('L0')
S1_draws = fit.stan_variable('S1')

L0 = L0_draws[0]
S1 = S1_draws[0]

df_L0 = pd.DataFrame(L0, index=utterances, columns=states)
df_S1 = pd.DataFrame(S1, index=states, columns=utterances)

print("L0:")
print(df_L0.to_string(float_format="{:.2f}".format))
print("S1:")
print(df_S1.to_string(float_format="{:.2f}".format))

L0:
       big_blue  big_red  small_blue
big        0.44     0.44        0.11
small      0.17     0.17        0.67
blue       0.50     0.01        0.50
red        0.01     0.98        0.01
S1:
            big  small  blue  red
big_blue   0.03   0.00  0.97 0.00
big_red    0.00   0.00  0.00 1.00
small_blue 0.00   1.00  0.00 0.00


In [ ]:
# Exercise 3.2.3
# The heat map generated by this cell shows P("blue"|"big_blue") for S1 given different size and color parameters.

import matplotlib.pyplot as plt

# This function was added (out of frustration) because the heat map just would not work at first.
# It is (as a beginner) not always intuitive how indexing works in Stan.
# The first versions of this cell generated heat maps that were only one color.
# Printing out the matrix showed that we read the indices in a confused manner. 
# This is possibly bloated, but it makes the code below more readable. 
def extract_matrix(samples, prefix, n_row, n_col):
    M = np.zeros((n_row, n_col))
    for r in range(1, n_row + 1):
        for c in range(1, n_col + 1):
            col = f"{prefix}[{r},{c}]"
            M[r - 1, c - 1] = samples[col].mean()
    return M

# We want to run the model for different values. This function will later be called from inside a loop to accomplish just that.
def run_rsa_for(x_size, x_color, rsa_model, base_stan_data):
    # The base data is copied to avoid potential side effects.
    stan_data = base_stan_data.copy()
    
    # This is familiar from the code for exercise 3.2.2.
    # The different values we want to compare will be plugged in here.
    stan_data["x_size"] = x_size
    stan_data["x_color"] = x_color

    # Same setup as in exercise 3.2.2
    fit = rsa_model.sample(
        stan_data,
        show_console=False,
        chains=1,            
        iter_warmup=200,
        iter_sampling=200,
        adapt_engaged=True
    )
    
    # Extract transformed parameters
    samples = fit.draws_pd()
    
    # Return mean values of L0 and S1.
    # Printing these out we get "L0[1,2]", "S1[2,3]", and so on and so forth.
    # This was helpful for troubleshooting, like the Stan manual suggests.
    L0_cols = [c for c in samples.columns if c.startswith("L0[")]
    S1_cols = [c for c in samples.columns if c.startswith("S1[")]

    U = stan_data["U"]
    S = stan_data["S"]

    L0_mean = extract_matrix(samples, "L0", U, S)   # U × S
    S1_mean = extract_matrix(samples, "S1", S, U)   # S × U

    return L0_mean, S1_mean

# We will try the values 0.1, 02..0.9, 1.0 for size and color.
# This simply seemed like the obvious choice, and it results in an interesting visualization.
x_size_vals = np.linspace(0.1, 1.0, 10)
x_color_vals = np.linspace(0.1, 1.0, 10)

# Here we store the results for visualization.
results = {}

# This is the loop announced above that runs the model for each value-pair.
for xs in x_size_vals:
    for xc in x_color_vals:

        # Rebuild meaning matrix
        # This is the same code as for exercise 3.2.2, except that it includes the variables xs and xc.
        # Otherwise we would not actually iterate over different value-pairs.
        meaning_matrix = np.zeros((n_utterances, n_states))
        for u, utterance in enumerate(utterances):
            for s, state in enumerate(states):
                extension = utterance in state
                if utterance in size_utterances:
                    x = xs
                elif utterance in color_utterances:
                    x = xc
                meaning_matrix[u, s] = x if extension else (1 - x)

        # Update stan_data
        stan_data["meaning_matrix"] = meaning_matrix.tolist()
        
        # Run RSA
        L0, S1 = run_rsa_for(xs, xc, rsa_model, stan_data)
        results[(xs, xc)] = S1

# We could plot any utterance.
utterance_index = utterances.index("blue")  
# We could plot any state
state_index = states.index("big_blue")      

# Standard heat map. 
# ChatGPT was used here via the following prompt: "Give me Python code to generate a heat map for 10 value pairs".
# Inserting our own variables, labels, and description resulted in the code below, which worked without further modifications.
# (As mentioned above, the heat map only contained one color at first, but that was because we had read in the matrix wrong.)
heatmap = np.zeros((len(x_size_vals), len(x_color_vals)))

for i, xs in enumerate(x_size_vals):
    for j, xc in enumerate(x_color_vals):
        heatmap[i, j] = results[(xs, xc)][state_index, utterance_index]

plt.figure(figsize=(8,6))
plt.imshow(heatmap, origin="lower", 
           extent=[min(x_color_vals), max(x_color_vals), 
                   min(x_size_vals), max(x_size_vals)],
           aspect="auto")
plt.colorbar(label=f"P(Says '{utterances[utterance_index]}' | state='{states[state_index]}')")
plt.xlabel("x_color")
plt.ylabel("x_size")
plt.title("Speaker S1 probability heatmap")
plt.show()

# Exercise 3.2.3 Result Analysis and Interpretation

The heat map shows what we expected it to. 
Let's look at the example P("blue"|"big_blue") for S1. (Other utterances/states result in analoguous heat maps.*)
The probability of saying "blue" increases with x_color. It decreases at the margins where x_size is very small or large.
This is in line with what we discussed in 3.2.2. A more extreme color value places more importance on a color term, of course.
If the color in "big_blue" is maximally important, S1 will always say "blue". When size is also (near) maximally important,
P("blue") is reduced because the size term might be used instead.

*Now on to what we mean by "analoguous", in which we will describe the distribution for every pair and why it's as expected:

Note that for every value-pair, entropy is maximal when they are both 0.5.

For P("blue"|"big_blue"), a sort of triangle shape emerges.
P(blue|big_blue) increases with higher x_color and x_size closer to 0.5, so the central right edge of the map concentrates P.

P(big|big_blue) basically rotates the shape by -90 degrees. P(big) increases with size and x_color closer to 0.5

P(red|big_red) has a different shape. Because "red" only appears in this utterance, P(red) is maximal for x_color >= 0.8.
When x_color = 0.6, P(red) is still maximal for x_size values close to 0.5. As with other value-pairs, more extreme size values
shift the importance toward the size term, reducing P(red) toward the fringes. x_color values below 0.5 result in the size
term being used instead.

P(big|big_red) displays a far narrower version of the shape P(big|big_blue) displayed. Probability is maximal when 
x_size >= 0.7 and when x_color is close to 0.5. The same explanation as above holds: Extreme x_color values mean that the
color term will be used instead of the size term. "red" only appears in one utterance, which obviously reduces its prior
probability and narrows the range of values where it is likely to be chosen by S1.

P(big|small_blue) has the shape of P(red|big_read) rotated by 90 degrees. It is maximal when x_size is small. When x_size
is close to 0.5, x_color has to be as well. This is expected because "big" does not describe "small_blue" well, so little
importance has to be placed on size in order for it to be selected. If size is closer to 0.5, extreme color values will lead
to a color term being chosen instead.

P(blue|small_blue) is the shape of P(big|big_red) rotated by 90 degrees: A narrow distribution with max. P when x_color is
large and x_size is close to 0.5. Again, high importance placed on color will lead to "blue" being selected, but it would be
rational to say "small" in this context, so even slightly more extreme x_size values will lead to "smalL" being said instead.

P(blue|big_red) -> like P(big|small_blue) rotated by 90 degrees. "blue" does not occur in the state "big_red", so extremely
low x_color values are the only viable way for it to be chosen.

P(red|big_blue) -> probable when x_color is minimal (because of course red != blue) and when x_size is closer to 0.5 (because
more extreme size values will favor using a size term instead) -> broad triangle shape as we've already seen above

P(red|small_blue) -> narrow version of P(red|big_blue). Only viable when x_color is minimized and x_size is unimportant 
because it is the wrong color and any more extreme size value will favor a size term. 
 
=> To sum up, three shapes emerge on the heat maps in line with our expectations. 
Unfit utterances are only chosen when the value of their category is minimized.
Fitting utterances are probable when their category value is large and the "competing" category value is not too extreme.
-> For fitting utterances a triangle shape emerges. 

The triangle is narrow for P(big|big_red) and P(red|small_blue) because "red" only appears in one state, therefore the size
of the competing category value does not need to be as high to reduce its probability. For P(blue|small_blue) a similar 
picture emerges because size is the category that immediately distinguishes the target from all other states, i.e. there
is only one small blue pin.
Utterances are probable when their category is given importance. Extreme values for competing categories reduce their 
probability. Narrow distributions reflect the fact that categories have different probabilities a priori. The distributions
as seen on our heat maps thus successfully indicate the areas that rationally chosen utterances fall into.

# Exercise 3.2.4 Construct a context where color is sufficient to mention and interpret the output.

(We basically did this in 3.2.3 by looking at every utterance/target pair.)

So far we worked with the following states: ["big_blue", "big_red", "small_blue"]
In the trial pictured (at this point far) above, the target is the small blue pin.
We could simply modify the trial by defining the states as ["big_red", "small_red", "small_blue"] and still targeting
"small_blue", where the color "blue" is as unambiguous as the size "small" is in the non-modified trial.

This modification does not change any of the probability distributions. All the patterns described in the answer to 3.2.3
reoccur. 
Uttering "blue" in this context is maximally probable when x_color is maximized and only influenced by x_size when the color
values are closer to 0.5. This is because color is so informative. It is the same pattern as for, e.g., P(red|big_red) in 3.2.3.
As in 3.2.3, P(small|small_blue) in the new context where color is sufficient results in a narrow triangular distribution
because even slightly extreme x_color values beat size. Again because color is so informative here.
Lastly, P(red|small_blue) inverts the shape of P(blue|small_blue). Because red is unfit, x_color has to be minimized for it
to be selected.

### 3.3) Model evaluation by comparison to experiment data (42 points)
3.3.1) Create a new file `sem_rsa.stan`. Adapt the vanilla RSA model in a way that allows you to infer all free parameters instead of specifying them beforehand. Condition the model on the observed production data (`data/data_exp1.csv`) and integrate over the free parameters. Preprocess the observed data in a way that you see fit for the modeling purpose. Assume uniform priors for each parameter. Use the generated quantities block in your Stan model to generate the posterior predictive distribution (read up [Stan documentation](https://mc-stan.org/docs/stan-users-guide/posterior-prediction.html) for this). Choose an appropriate number of iterations for warm up and sampling from the posterior. (16 points)

    `stan/sem_rsa.stan` samples the pragmaticity parameter `alpha` as well as two cost parameters keyed to size and color indicators. The preprocessing cell builds the graded meaning matrix, utterance-specific indicator vectors, and per-trial state/utterance IDs from `data_exp1.csv`, so the Stan program can infer all three parameters jointly and generate posterior predictive utterances.


In [2]:
# Preprocess experiment data for the semantic RSA model
# Load raw production data from the TSV file
rsa_df = pd.read_csv('data_exp1.csv', sep='	')

# Map the three utterance classes onto integer IDs and drop the rest
utterance_map = {'size': 1, 'color': 2, 'size and color': 3}
rsa_df = rsa_df[rsa_df['UtteranceType'].isin(utterance_map)].reset_index(drop=True)
rsa_df['utt_id'] = rsa_df['UtteranceType'].map(utterance_map).astype(int)

# Build a unique state key so we can collapse identical contexts
state_key_cols = [
    'TargetItem',
    'SufficientProperty',
    'RedundantProperty',
    'NumDistractors',
    'NumSameDistractors',
    'SceneVariation',
]
rsa_df['state_key'] = rsa_df[state_key_cols].astype(str).agg(' | '.join, axis=1)
state_keys = rsa_df['state_key'].unique()
state_lookup = {key: idx + 1 for idx, key in enumerate(state_keys)}
rsa_df['state_id'] = rsa_df['state_key'].map(state_lookup).astype(int)

U = len(utterance_map)
S = len(state_lookup)
N = len(rsa_df)

# Keep the alphabetized target list for later grouping in the analysis
target_names = sorted(rsa_df['TargetItem'].unique())

# Precompute binary indicators for whether an utterance mentions size and/or color
ordered_utts = sorted(utterance_map.items(), key=lambda kv: kv[1])
category_lookup = {
    'size': (1, 0),
    'color': (0, 1),
    'size and color': (1, 1),
}
size_indicator = np.zeros(U, dtype=int)
color_indicator = np.zeros(U, dtype=int)
for utt, idx in ordered_utts:
    size_indicator[idx - 1], color_indicator[idx - 1] = category_lookup[utt]

# Columns that list the distractor object names
alt_cols = [f'alt{i}Name' for i in range(1, 5)]
epsilon = 1e-3 # so that we don't get zeros


# Parse strings such as 'small_blue_belt' into size/color attributes
def parse_item(name):
    if pd.isna(name) or name == 'undefined':
        return None
    parts = str(name).split('_')
    if len(parts) < 3:
        return None
    return {'size': parts[0], 'color': parts[1]}


# Gather the target plus distractors for each trial
def collect_objects(row):
    objs = []
    target = parse_item(row['TargetItem'])
    if target is None:
        raise ValueError(f"Missing target description for row {row.name}")
    objs.append(target)
    for col in alt_cols:
        item = parse_item(row.get(col))
        if item is not None:
            objs.append(item)
    num_required = int(row['NumDistractors']) + 1
    if len(objs) < num_required:
        raise ValueError(f"Context {row.name} expected {num_required} objects, got {len(objs)}")
    return objs[:num_required]


# Compute graded semantics scores per context from the object lineup
def semantics_from_context(row):
    objs = collect_objects(row)
    target = objs[0]
    total = len(objs)
    distractors = max(total - 1, 1)

    size_matches = sum(1 for obj in objs if obj['size'] == target['size'])
    color_matches = sum(1 for obj in objs if obj['color'] == target['color'])
    both_matches = sum(
        1
        for obj in objs
        if obj['size'] == target['size'] and obj['color'] == target['color']
    )

    size_conf = (size_matches - 1) / distractors
    color_conf = (color_matches - 1) / distractors
    both_conf = (both_matches - 1) / distractors

    size_score = 1.0 - size_conf
    color_score = 1.0 - color_conf
    both_score = 1.0 - both_conf

    if row['SufficientProperty'] == 'size':
        size_score = min(1.0, size_score + 0.15)
    else:
        color_score = min(1.0, color_score + 0.15)

    if 'size' in row['RedundantProperty']:
        size_score = max(0.05, size_score - 0.1)
    if 'color' in row['RedundantProperty']:
        color_score = max(0.05, color_score - 0.1)

    both_score = np.clip(both_score + max(size_conf, color_conf) * 0.5, 0.0, 1.0)

    return (
        # make sure there are no zeros 
        np.clip(size_score, 0.05, 1.0),
        np.clip(color_score, 0.05, 1.0),
        np.clip(both_score, 0.05, 1.0),
    )


# Build the meaning matrix once per unique state using the helper above
meaning_matrix = np.zeros((U, S), dtype=float)
state_reps = rsa_df.drop_duplicates('state_key', keep='first')

for _, state_row in state_reps.iterrows():
    sid = state_lookup[state_row['state_key']] - 1
    size_score, color_score, both_score = semantics_from_context(state_row)
    meaning_matrix[utterance_map['size'] - 1, sid] = epsilon + (1 - epsilon) * size_score
    meaning_matrix[utterance_map['color'] - 1, sid] = epsilon + (1 - epsilon) * color_score
    meaning_matrix[utterance_map['size and color'] - 1, sid] = epsilon + (1 - epsilon) * both_score

# Sanity checks for Stan compatibility
assert meaning_matrix.shape == (U, S)
assert meaning_matrix.min() > 0.0

# Hand everything to Stan, including the new indicator arrays
stan_data_sem = {
    'S': S,
    'U': U,
    'N': N,
    'meaning_matrix': meaning_matrix.tolist(),
    'size_indicator': size_indicator.astype(int).tolist(),
    'color_indicator': color_indicator.astype(int).tolist(),
    'state_id': rsa_df['state_id'].to_numpy(dtype=int).tolist(),
    'utt_id': rsa_df['utt_id'].to_numpy(dtype=int).tolist(),
}

# Log output
print(f'Prepared {N} trials spanning {S} unique contexts.')
print('meaning_matrix shape:', meaning_matrix.shape)
print('Size indicator:', size_indicator.tolist())
print('Color indicator:', color_indicator.tolist())



Prepared 2076 trials spanning 1427 unique contexts.
meaning_matrix shape: (3, 1427)
Size indicator: [1, 0, 1]
Color indicator: [0, 1, 1]


In [11]:
size_score, color_score, both_score = semantics_from_context(state_row)
print(size_score)
print(color_score)
print(both_score)

0.05
1.0
1.0


In [12]:
sem_rsa_file = os.path.join('stan', 'sem_rsa.stan')
sem_rsa_model = CmdStanModel(stan_file=sem_rsa_file)

In [13]:
sem_inits = [{'alpha': 2.0, 'cost_size': 0.5, 'cost_color': 0.5} for _ in range(4)]

sem_fit = sem_rsa_model.sample(
    data=stan_data_sem,
    chains=4,
    iter_warmup=400,
    iter_sampling=400,
    seed=123,
    inits=sem_inits,
    show_progress=True,
)


21:04:16 - cmdstanpy - INFO - CmdStan start processing
chain 1:   0%|          | 0/800 [00:00<?, ?it/s, (Warmup)]


chain 1:  12%|█▎        | 100/800 [00:04<00:32, 21.49it/s, (Warmup)]


chain 1:  38%|███▊      | 300/800 [00:07<00:10, 48.74it/s, (Warmup)]


chain 1:  50%|█████     | 400/800 [00:08<00:07, 55.13it/s, (Sampling)]








chain 1:  62%|██████▎   | 500/800 [00:10<00:05, 52.87it/s, (Sampling)]


chain 1:  75%|███████▌  | 600/800 [00:13<00:04, 48.96it/s, (Sampling)]


chain 1:  88%|████████▊ | 700/800 [00:15<00:02, 46.13it/s, (Sampling)]


chain 1: 100%|██████████| 800/800 [00:18<00:00, 42.28it/s, (Sampling)]


chain 2: 100%|██████████| 800/800 [00:19<00:00, 40.56it/s, (Sampling completed)]

chain 3: 100%|██████████| 800/800 [00:19<00:00, 40.56it/s, (Sampling completed)]


chain 4: 100%|██████████| 800/800 [00:19<00:00, 40.57it/s, (Sampling completed)]


21:04:36 - cmdstanpy - INFO - CmdStan done processing.
21:04:36 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: categorical_lpmf: Probabilities parameter is not a valid simplex. sum(Probabilities parameter) = -nan, but should be 1 (in 'sem_rsa.stan', line 58, column 4 to column 46)
Exception: categorical_lpmf: Probabilities parameter is not a valid simplex. sum(Probabilities parameter) = -nan, but should be 1 (in 'sem_rsa.stan', line 58, column 4 to column 46)
Exception: categorical_lpmf: Probabilities parameter is not a valid simplex. sum(Probabilities parameter) = -nan, but should be 1 (in 'sem_rsa.stan', line 58, column 4 to column 46)
Exception: categorical_lpmf: Probabilities parameter is not a valid simplex. sum(Probabilities parameter) = -nan, but should be 1 (in 'sem_rsa.stan', line 58, column 4 to column 46)
Consider re-running with show_console=True if the above output is unclear!


In [14]:
diagnostics = sem_fit.diagnose()
print(diagnostics)

sem_summary = sem_fit.summary()
params_of_interest = sem_summary.loc[
    ['alpha', 'cost_size', 'cost_color'],
    ['Mean', 'StdDev', 'R_hat', 'ESS_bulk', 'ESS_tail'],
]
params_of_interest


Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Rank-normalized split effective sample size satisfactory for all parameters.

Rank-normalized split R-hat values satisfactory for all parameters.

Processing complete, no problems detected.



,Mean,StdDev,R_hat,ESS_bulk,ESS_tail
alpha,2.828990,0.102537,1.00217,1437.52,1141.820
cost_size,0.749837,0.068854,1.00142,1491.38,1318.370
cost_color,0.005514,0.005763,1.00081,1234.92,737.077


3.3.2) Diagnose the model convergence and take actions if necessary. (4 points)

    `diagnose()` reported no divergences, treedepth hits, or BFMI problems for the 4×(400 warmup + 400 sample) run. The largest R-hat across the three tuned parameters was ≈1.002 with bulk ESS well above 1k, so no further tuning was necessary despite the occasional simplex warning.

3.3.3) Interpret a summary of the fitted model. (6 points)

    Posterior means are `alpha ≈ 2.83 ± 0.10`, `cost_size ≈ 0.75 ± 0.07`, and `cost_color ≈ 0.006 ± 0.006`. Speakers therefore behave slightly pragmatically, strongly penalize size modifiers, and treat color mentions as essentially free. Redundant utterances inherit both penalties, but because color mentions barely change the cost for redundant utterances (compared to just size utterances), overinformative utterances will be favoured for informativeness reasons: When size should be mentioned, color can be addded "for free".
    For comparison, the single-cost run produced `alpha ≈ 3.17` with `cost_weight ≈ 0.028`, which effectively removed any penalty for redundancy.


In [15]:
S1_draws = sem_fit.stan_variable('S1')
S1_mean = S1_draws.mean(axis=0)

redundant_idx = utterance_map['size and color'] - 1
state_indices = rsa_df['state_id'].to_numpy(dtype=int) - 1
posterior_per_trial = S1_mean[state_indices, redundant_idx]

posterior_overinform = (
    rsa_df.assign(posterior_overinform=posterior_per_trial)
          .groupby('TargetItem')['posterior_overinform']
          .mean()
          .reindex(target_names)
          .to_numpy()
)

empirical_overinform = (
    rsa_df.assign(overinform=rsa_df['UtteranceType'] == 'size and color')
          .groupby('TargetItem')['overinform']
          .mean()
          .reindex(target_names)
          .to_numpy()
)

corr = np.corrcoef(empirical_overinform, posterior_overinform)[0, 1]
comparison_df = pd.DataFrame(
    {
        'target': target_names,
        'empirical_overinform': empirical_overinform,
        'posterior_mean_overinform': posterior_overinform,
    }
)
print(comparison_df.head())
print(f'Pearson r between empirical and posterior means: {corr:.3f}')


              target  empirical_overinform  posterior_mean_overinform
0  big_black_avocado              0.200000                   0.176802
1     big_black_belt              0.466667                   0.176300
2     big_black_book              0.181818                   0.147508
3     big_black_comb              0.076923                   0.152157
4   big_black_turtle              0.187500                   0.161609
Pearson r between empirical and posterior means: 0.262


3.3.4) Correlate the model's posterior predictive distribution for overinformative utterance probabilities with the empirical data to assess and interpret model fit to the data. (8 points)

    With the two cost parameters the Pearson r between empirical and posterior redundant rates climbs to ≈0.26 (up from ≈−0.30 under the single-cost model). The model now recovers some of the ordering across targets, but the signal is still weak compared with the empirical variability. The model’s predictions are more compressed and never reach the high empirical values for some objects. It captures part of the structure in the data, but it still underestimates the strength of the item-specific differences.

3.3.5) Bonus: Introduce separate cost parameters for size and color. (6 bonus points)

    The single cost parameter was replaced with two non-negative cost parameters, cost_size and cost_color, and a binary indicators was added, telling Stan whether each utterance mentions size, color, or both. The redundant utterance (“size and color”) pays both costs.
    In the posterior, cost_size is clearly larger (around 0.75), while cost_color is very close to zero (around 0.006). This means the model learns that size words are expensive, but adding a color word is almost free. With only one cost parameter, these two effects were forced to be the same and got averaged into a small overall cost.

3.3.6) Interpret and discuss your findings. (8 points)

    The parameter alpha shows that speakers care about informativeness, but the key result is the strong asymmetry in the inferred costs. Size words carry most of the cost, while color words are treated as very cheap.
    As a result, the model is happy to add a color modifier on top of a size description, because it increases or maintains informativeness at almost no extra cost. This helps the model better match the observed high rate of “overinformative” size-and-color utterances and improves the fit compared to the single-cost version.

## X) Reflection (no points, but mandatory)

Reflect on your group work. What went well? What did not go well? 

Please note down the group members' team roles anonymously and reflect on how you filled this role.

- Our team made up of an Implementer, Plant, and Monitor-Evaluator proved an adequate meshup. The implementer indeed implemented a large proportion of the solutions, although everyone rose to the occasion. We had a fruitful vibe, so the plant prosociality paid off. The Monitor-Evaluator did evaluate too, but could have pulled their weight a little more. Our initial division of labor turned out to be sensible, but it took us some time to figure out an approach to the last exercise. A more extensive early group work phase could have made this part less chaotic and the deadline less stress-inducing. 

- Generally, we expect that the division of labor as well as the different stenghts of everyone will become more relevant when we have the the bigger final project to work on. This homework is a good testbed though.   